# Dritte Übung

Erst lese ich alle Dateine ein und speichere immer den Namen der Datei und den Text als Liste von tokenisierten Texten:

In [1]:
import glob
import codecs
import nltk
from HanTa import HanoverTagger as ht


tagger = ht.HanoverTagger('morphmodel_ger.pgz')

corpus = []

filelist = glob.glob("Firmen/*.txt")
for f in filelist:
    text = []
    datei = codecs.open(f,'r','utf8')
    for zeile in datei:
        zeile = zeile.strip('= \n')
        sentences = nltk.sent_tokenize(zeile,language='german')
        for satz in sentences:
            wortliste = nltk.word_tokenize(satz,language='german')
            text.append(wortliste)
    corpus.append((f[7:-4],text)) #Name ohne den Namen des Ordners und ohne die Endung.txt

    
    datei.close()

Schauen wir mal, wie ein Element aus der Liste aussieht:

In [6]:
corpus[167]

('Crocs',
 [['Als',
   'Crocs',
   'werden',
   'die',
   'vom',
   'US-amerikanischen',
   'Unternehmen',
   'Crocs',
   'Inc',
   '.'],
  ['vermarkteten', 'bunten', 'Kunststoffschuhe', 'bezeichnet', '.'],
  ['Auffallendstes',
   'Merkmal',
   'der',
   'Crocs',
   'sind',
   'die',
   'leuchtenden',
   'Farben',
   '.'],
  ['Neben',
   'Hellgrün',
   ',',
   'Rot',
   ',',
   'Gelb',
   'und',
   'Türkis',
   'gibt',
   'es',
   'aber',
   'auch',
   'dezentere',
   'Farben',
   '.'],
  ['Bis',
   'zu',
   '26',
   'Farben',
   'sind',
   'für',
   'die',
   'Hauptmodelle',
   'lieferbar',
   '.'],
  ['Material'],
  ['Das',
   'Schuhmaterial',
   'nennt',
   'sich',
   'PCCR',
   '(',
   'Proprietary',
   'closed-cell',
   'resin',
   ')',
   ',',
   'ist',
   'wasserfest',
   'und',
   'nicht',
   'porös',
   ',',
   'weshalb',
   'sich',
   'Bakterien',
   'und',
   'Pilze',
   'nicht',
   'so',
   'leicht',
   'festsetzen',
   ',',
   'was',
   'die',
   'Schuhe',
   'hygienischer

In [7]:
from collections import Counter

def count_nouns(text):
    nouns = []
    for satz in text:
        lemmata = tagger.tag_sent(satz)
        for (word,lemma,pos) in lemmata:
            if pos == 'NN' or pos == 'NE':
                nouns.append(lemma)
                
    nouns_cnt = Counter(nouns)
    return nouns_cnt.most_common()

In [8]:
name,text = corpus[468]
substantive = count_nouns(text)
print(name,substantive[:10])

Palfinger [('Palfinger', 11), ('Unternehmen', 5), ('Prozent', 5), ('Jahr', 3), ('Bereich', 3), ('Hersteller', 2), ('Krane', 2), ('Produkt', 2), ('Modell', 2), ('Marktanteil', 2)]


## Jetzt mit IDF

In [9]:
df = Counter()

for name,text in corpus:
    substantive = [wort for wort,_ in count_nouns(text)]
    df.update(substantive)

In [10]:
df.most_common(10)

[('Unternehmen', 611),
 ('Einzelnachweise', 578),
 ('Weblinks', 549),
 ('Jahr', 468),
 ('Geschichte', 385),
 ('Mitarbeiter', 326),
 ('Sitz', 302),
 ('Deutschland', 293),
 ('%', 283),
 ('Million', 264)]

In [11]:
import math 

idf = {}
for w in df:
    idf[w] = math.log(len(corpus)/df[w])

In [12]:
idf['Unternehmen']

0.12593304001830796

In [13]:
idf['Schuh']

4.461588457510067

In [17]:
def tfidf_nouns(text):
    nouns = []
    for satz in text:
        lemmata = tagger.tag_sent(satz)
        for (word,lemma,pos) in lemmata:
            if pos == 'NN' or pos == 'NE':
                nouns.append(lemma)
                
    nouns_cnt = Counter(nouns)
    nouns_tfidf = Counter()
    for w in nouns_cnt:
        nouns_tfidf[w] = nouns_cnt[w] * idf[w]
    
    return nouns_tfidf.most_common()

In [18]:
name,text = corpus[468]
substantive_freq = count_nouns(text)
substantive_idf = tfidf_nouns(text)
print(name)
print(substantive_freq[:10])
print(substantive_idf[:10])

Palfinger
[('Palfinger', 11), ('Unternehmen', 5), ('Prozent', 5), ('Jahr', 3), ('Bereich', 3), ('Hersteller', 2), ('Krane', 2), ('Produkt', 2), ('Modell', 2), ('Marktanteil', 2)]
[('Palfinger', 71.95132999108894), ('Kranhersteller', 13.082059998379806), ('Ndm', 13.082059998379806), ('Containerwechselsysteme', 13.082059998379806), ('Mitnahmestapler', 13.082059998379806), ('Hubarbeitsbühnen', 13.082059998379806), ('Eisenbahn-systemlösung', 13.082059998379806), ('Epsilon', 13.082059998379806), ('Ladebordwände', 11.695765637259916), ('Krane', 10.884835421043586)]
